In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/mysql-connector-java-5.1.45-bin.jar pyspark-shell'
os.environ["SPARK_HOME"] = "/Users/kunalsharma/Downloads/spark-2.4.4-bin-hadoop2.7"
os.environ["PYSPARK_PYTHON"]="/usr/local/bin/python3"

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from zipfile import ZipFile 
import mysql.connector
import sys 

In [31]:
def part1(tablename, path, filename):
    # Create table in mySql 
    mycursor.execute("CREATE TABLE IF NOT EXISTS "+tablename+" (credit_card_number varchar(50), ipv4 varchar(50))")
    
    # Read file into spark dataframe   
    fraud_df = read_zipfile(path,filename)

    # Write the dataframe into Mysql test table
    fraud_df.write.format('jdbc').options(url='jdbc:mysql://localhost/Al',
                                         driver='com.mysql.jdbc.Driver',
                                         dbtable=tablename,
                                         user='root',
                                         password = '1234').mode('overwrite').save()
    
    fraud_df.show()
    return fraud_df

In [3]:
# Found nulls for state names without header in the file, created schema to include data

def part1_extra(tablename, path, filename):
    # Create table in mySql 
    mycursor.execute("CREATE TABLE IF NOT EXISTS "+tablename+" (credit_card_number varchar(50), ipv4 varchar(50), state varchar(10))")
    
    # Make a schema 
    schema = StructType([
        StructField("credit_card_number", StringType(), True),
        StructField("ipv4", StringType(), True),
        StructField("state", StringType(), True)])
    
    # Read file into spark dataframe  
    fraud_df = read_zipfile_withschema(path,filename, schema)
 
    # Write the dataframe into mysql testState table
    fraud_df.write.format('jdbc').options(
        driver = 'com.mysql.jdbc.Driver'
        ,url=jdbc_url
        ,dbtable=tablename ).mode('overwrite').save()
    
    fraud_df.show()
    return fraud_df

In [4]:
def part2(path, filename):
    # Regular expression(key) of vendors(value) in dictionary
    expressions = {"^4": "Visa", 
           "^(5018|5020|5038|56)": "Maestro",
           "^5[0-6]": "Mastercard", 
           "^(36|38|30[0-1]|30[4-5])": "Diners Club", 
           "^(6011|65)": "Discover", 
           "^35": "JCB16", 
           "^(2131|1800)" : "JCB15",
           "^(34|37)": "American Express" }
    
    # Read file into spark dataframe  
    trans_df = read_zipfile(path,filename)
    
    # Sanitize the data in each dataframe using regular expression in the dictionary   
    sanitized_trans_df = trans_df.filter(trans_df.credit_card_number.rlike("|".join(list(expressions.keys()))))
    sanitized_trans_df.show()
    return sanitized_trans_df


In [5]:
# While sanitizing the data, categorize each row with its vendor corresponding to the card number 

def part2_extra(path, filename):
    import re
    from pyspark.sql.functions import lit

    # Regular expression(key) of vendors(value) in dictionary
    expressions = {"^4": "Visa", 
           "^(5018|5020|5038|56)": "Maestro",
           "^(51|52|54|55|222)": "Mastercard", 
           "^(36|38|30[0-1]|30[4-5])": "Diners Club", 
           "^(6011|65)": "Discover", 
           "^35": "JCB16", 
           "^(2131|1800)" : "JCB15",
           "^(34|37)": "American Express" }
    
    # Read file into spark dataframe  
    trans_df = read_zipfile(path,filename)
    
    # Sanitize the data in each dataframe using regular expression in the dictionary   
    sanitized_trans_df = trans_df.filter(trans_df.credit_card_number.rlike("|".join(list(expressions.keys()))))
    
    categorized_trans_df = sanitized_trans_df.withColumn('vendor',lit(0))

    vendorslist = list(expressions.keys())
    for v in vendorslist:
        categorized_trans_df = categorized_trans_df.withColumn('vendor', 
                    F.when(F.col('credit_card_number').rlike(v),expressions.get(v))
                               .otherwise(F.col('vendor')))
    categorized_trans_df.show()    
    return categorized_trans_df


In [6]:
def part3(df1, df2):
    df = df1.join(df2, df1.credit_card_number == df2.credit_card_number, 'inner').drop(df1.credit_card_number).drop(df1.ipv4)
    df.show()
    return df,df.count()

def part3_state(df):
    return df.groupBy("state").count().sort("count",ascending=False)


def part3_vendor(df):
    return df.groupBy("vendor").count().sort("count",ascending=False)


In [7]:
# Save the dataframe with 3 columns; credit_card_number, ipv4, state into JSON

def part3_savejson(df, path):
    df.createOrReplaceTempView("FraudulentTransactions")
    result = spark.sql("SELECT credit_card_number, ipv4, state FROM FraudulentTransactions")
    result = result.withColumn('credit_card_number',F.regexp_replace('credit_card_number', '\d{9}$', '*********'))
    
    size_list_udf = F.udf(lambda data: sys.getsizeof(data))
    result = result.withColumn('byte',size_list_udf(F.col('credit_card_number'))+size_list_udf(F.col('ipv4'))+size_list_udf(F.col('state')))
    result.coalesce(1).write.mode('overwrite').option("header", "true").json(path)

    return result

# Save the dataframe with 3 columns; credit_card_number, ipv4, state into orc

def part3_savebinary(df, path):
    df.createOrReplaceTempView("FraudulentTransactions")
    result = spark.sql("SELECT credit_card_number, ipv4, state FROM FraudulentTransactions")
    result = result.withColumn('credit_card_number',F.regexp_replace('credit_card_number', '\d{9}$', '*********'))
    
    size_list_udf = F.udf(lambda data: sys.getsizeof(data))
    result = result.withColumn('byte',size_list_udf(F.col('credit_card_number'))+size_list_udf(F.col('ipv4'))+size_list_udf(F.col('state')))
    
    result.repartition(1).write.mode('overwrite').option("header", "true").format("orc").save(path)
    return result

In [8]:
def read_zipfile(path,filename):
    
    with ZipFile(path, 'r') as zip: 
        # printing all the contents of the zip file 
        zip.printdir() 
        df = spark.read.option("header", "true").csv(zip.extract(filename))
    return df

In [9]:
def read_zipfile_withschema(path,filename,schema):
    
    with ZipFile(path, 'r') as zip: 
        # printing all the contents of the zip file 
        zip.printdir() 
        df = spark.read.option("header", "true").csv(zip.extract('fraud'), schema=schema)
    return df

In [10]:
if __name__ == '__main__':
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="1234",
        database="Al",
        auth_plugin='mysql_native_password')
    
    mycursor = mydb.cursor()
    
    # mySql Properties (for read/write spark dataframe)
    hostname = "localhost" 
    dbname = "Al"
    username = "root"
    password = "1234"
    jdbc_url = "jdbc:mysql://{0}/{1}?user={2}&password={3}".format(hostname, dbname,username,password)
    
    # Load relevant objects
    sc = SparkContext('local')
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)
    

In [15]:
path = "/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/datasets/"
df1 = part1("fraud", path+"fraud.zip", "fraud")
part1_extra("fraud2", path+"fraud.zip", "fraud")

File Name                                             Modified             Size
fraud                                          2019-01-14 20:51:54        31227
__MACOSX/                                      2019-01-14 20:52:06            0
__MACOSX/._fraud                               2019-01-14 20:51:54          400
+-------------------+---------------+
| credit_card_number|           ipv4|
+-------------------+---------------+
|      4013901026491|  172.23.51.228|
|    345936465222676|  192.168.102.7|
|   4829769584081989|192.168.246.102|
|     30340825930914| 192.168.222.52|
|4302964471893676356|   10.84.64.210|
|   4897487775841494| 192.168.188.11|
|   3504134000927360| 172.17.225.208|
|    343568314271448|  10.159.127.41|
|     36795457706136| 192.168.85.173|
|    213105387849185|192.168.249.162|
|   4158823194013915| 192.168.19.169|
|   3557121809488039| 192.168.99.101|
|   4411710233936193|192.168.248.223|
|   3593952520772269|192.168.189.168|
|       562632689286|  10.198.220.

DataFrame[credit_card_number: string, ipv4: string, state: string]

In [16]:
df2_001 = part2(path+"transaction-001.zip", "transaction-001")
df2_002 = part2(path+"transaction-002.zip", "transaction-002")

File Name                                             Modified             Size
transaction-001                                2019-01-14 20:50:50      8415774
__MACOSX/                                      2019-01-14 20:52:12            0
__MACOSX/._transaction-001                     2019-01-14 20:50:50          176
+-------------------+---------------+-----+
| credit_card_number|           ipv4|state|
+-------------------+---------------+-----+
|      4938684086769|  172.29.189.27|   CA|
|    213128820373753| 172.22.174.184|   AR|
|   3554681727155351|  172.24.27.121|   ID|
|      4339158798023|   10.87.17.223|   SC|
|    341265497366150|   10.134.18.19|   AR|
|    180000002766501| 172.28.140.142|   SD|
|     30500400570720| 172.22.198.128|   IA|
|   4360876516417234|  192.168.71.93|   KS|
|    213137333750142|  10.127.17.197|   VT|
|     36434057403932| 172.29.235.176|   LA|
|   4959809659477746|  172.26.163.69|   NJ|
|   3565210364933982|   10.135.99.42|   CO|
|4717807932215388194

In [17]:
df2_e001 = part2_extra(path+"transaction-001.zip", "transaction-001")
df2_e002 = part2_extra(path+"transaction-002.zip", "transaction-002")

File Name                                             Modified             Size
transaction-001                                2019-01-14 20:50:50      8415774
__MACOSX/                                      2019-01-14 20:52:12            0
__MACOSX/._transaction-001                     2019-01-14 20:50:50          176
+-------------------+--------------+-----+----------------+
| credit_card_number|          ipv4|state|          vendor|
+-------------------+--------------+-----+----------------+
|      4938684086769| 172.29.189.27|   CA|            Visa|
|    213128820373753|172.22.174.184|   AR|           JCB15|
|   3554681727155351| 172.24.27.121|   ID|           JCB16|
|      4339158798023|  10.87.17.223|   SC|            Visa|
|    341265497366150|  10.134.18.19|   AR|American Express|
|    180000002766501|172.28.140.142|   SD|           JCB15|
|     30500400570720|172.22.198.128|   IA|     Diners Club|
|   4360876516417234| 192.168.71.93|   KS|            Visa|
|    213137333750142

In [18]:
df2 = df2_e001.union(df2_e002)

In [19]:
df1.show()

+-------------------+---------------+
| credit_card_number|           ipv4|
+-------------------+---------------+
|      4013901026491|  172.23.51.228|
|    345936465222676|  192.168.102.7|
|   4829769584081989|192.168.246.102|
|     30340825930914| 192.168.222.52|
|4302964471893676356|   10.84.64.210|
|   4897487775841494| 192.168.188.11|
|   3504134000927360| 172.17.225.208|
|    343568314271448|  10.159.127.41|
|     36795457706136| 192.168.85.173|
|    213105387849185|192.168.249.162|
|   4158823194013915| 192.168.19.169|
|   3557121809488039| 192.168.99.101|
|   4411710233936193|192.168.248.223|
|   3593952520772269|192.168.189.168|
|       562632689286|  10.198.220.41|
|       675979363636| 172.21.115.184|
|   3506507615619043|   10.172.55.81|
|   3560888773289883|   10.186.58.92|
|   5128258542512744|  10.65.191.187|
|   4673436419900968|   10.68.149.54|
+-------------------+---------------+
only showing top 20 rows



In [20]:
df2.show()

+-------------------+--------------+-----+----------------+
| credit_card_number|          ipv4|state|          vendor|
+-------------------+--------------+-----+----------------+
|      4938684086769| 172.29.189.27|   CA|            Visa|
|    213128820373753|172.22.174.184|   AR|           JCB15|
|   3554681727155351| 172.24.27.121|   ID|           JCB16|
|      4339158798023|  10.87.17.223|   SC|            Visa|
|    341265497366150|  10.134.18.19|   AR|American Express|
|    180000002766501|172.28.140.142|   SD|           JCB15|
|     30500400570720|172.22.198.128|   IA|     Diners Club|
|   4360876516417234| 192.168.71.93|   KS|            Visa|
|    213137333750142| 10.127.17.197|   VT|           JCB15|
|     36434057403932|172.29.235.176|   LA|     Diners Club|
|   4959809659477746| 172.26.163.69|   NJ|            Visa|
|   3565210364933982|  10.135.99.42|   CO|           JCB16|
|4717807932215388194| 10.237.51.201|   WI|            Visa|
|4233048664267222289| 172.19.63.114|   F

In [21]:
df3,count = part3(df1, df2)
print(count)

+-------------------+---------------+-----+----------------+
| credit_card_number|           ipv4|state|          vendor|
+-------------------+---------------+-----+----------------+
|      4013901026491|  172.23.51.228|   NV|            Visa|
|   5128258542512744|  10.65.191.187|   NJ|      Mastercard|
|   4673436419900968|   10.68.149.54|   ND|            Visa|
|   6011531195808785| 192.168.216.58|   MA|        Discover|
|   3518829214836604|    10.55.8.196|   AK|           JCB16|
|       503886101024| 192.168.202.55|   MI|         Maestro|
|    346979098869785|    10.42.44.16|   VA|American Express|
|    213175362057328|  10.15.184.118|   AR|           JCB15|
|     36828949701611|  10.100.190.63|   CA|     Diners Club|
|   6011712584028793|   192.168.6.95|   NE|        Discover|
|   4408482692580798| 172.18.121.149|   VA|            Visa|
|   4716466790794127| 172.28.210.180|   MD|            Visa|
|   4581622575175591|192.168.161.187|   NH|            Visa|
|   4534971465733250|   

In [22]:
part3_state(df3).show()
part3_vendor(df3).show()


+-----+-----+
|state|count|
+-----+-----+
|   LA|   27|
|   AK|   26|
|   VA|   25|
|   WA|   25|
|   KS|   24|
|   KY|   23|
|   MS|   23|
|   MD|   23|
|   SC|   22|
|   PA|   22|
|   RI|   22|
|   VT|   21|
|   IN|   21|
|   MA|   20|
|   DE|   20|
|   OH|   20|
|   NY|   20|
|   DC|   20|
|   MN|   19|
|   NH|   19|
+-----+-----+
only showing top 20 rows

+----------------+-----+
|          vendor|count|
+----------------+-----+
|            Visa|  352|
|           JCB16|  185|
|        Discover|   99|
|American Express|   84|
|           JCB15|   74|
|      Mastercard|   55|
|     Diners Club|   54|
|         Maestro|   24|
+----------------+-----+



In [24]:
part3_savejson(df3,"/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/fraud_transaction.JSON").show()
part3_savebinary(df3,"/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/fraud_transaction.orc").show()

+-------------------+---------------+-----+-----+
| credit_card_number|           ipv4|state| byte|
+-------------------+---------------+-----+-----+
|      4013*********|  172.23.51.228|   NV|175.0|
|   5128258*********|  10.65.191.187|   NJ|178.0|
|   4673436*********|   10.68.149.54|   ND|177.0|
|   6011531*********| 192.168.216.58|   MA|179.0|
|   3518829*********|    10.55.8.196|   AK|176.0|
|       503*********| 192.168.202.55|   MI|175.0|
|    346979*********|    10.42.44.16|   VA|175.0|
|    213175*********|  10.15.184.118|   AR|177.0|
|     36828*********|  10.100.190.63|   CA|176.0|
|   6011712*********|   192.168.6.95|   NE|177.0|
|   4408482*********| 172.18.121.149|   VA|179.0|
|   4716466*********| 172.28.210.180|   MD|179.0|
|   4581622*********|192.168.161.187|   NH|180.0|
|   4534971*********|   10.30.201.50|   KY|177.0|
|    348623*********|  10.171.147.35|   DC|177.0|
|   3562843*********|    10.55.9.217|   NH|176.0|
|   3503861*********| 192.168.103.31|   NY|179.0|


# Unit Tests

In [29]:
# Unit Test 
import unittest2 as unittest
import logging

logging.getLogger('py4j.java_gateway').setLevel(logging.WARN)
logging.getLogger("py4j").setLevel(logging.WARN)

class PySparkTestCase(unittest.TestCase):
    """
    SparkContext being created for each test
    """
    partition_num = 1

    def setUp(self):
        # Setup a new spark context for each test
        class_name = self.__class__.__name__
        self.spark_session = SparkSession.builder \
            .master("local") \
            .appName(class_name) \
            .config("spark.ui.showConsoleProgress", False) \
            .getOrCreate()
    
        self.sc = self.spark_session.sparkContext
        self.spark = SparkSession(self.sc)
        
        # JDBC mySql  Properties (for read/write spark dataframe)
        hostname = "localhost" 
        dbname = "Al"
        username = "root"
        password = "1234"
        jdbc_url = "jdbc:mysql://{0}/{1}?user={2}&password={3}".format(hostname, dbname,username,password)

        # Solutions
        self.part1_sol = self.spark.read.option("header", "true").csv("/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/part1solution")
        self.part1extra_sol = self.spark.read.option("header", "true").csv("/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/part1extrasolution")

        self.part2_sol = self.spark.read.option("header", "true").csv("/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/part2solution")
        self.part2extra_sol = self.spark.read.option("header", "true").csv("/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/part2extrasolution")

        self.part3_sol = self.spark.read.option("header", "true").csv("/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/part3solution")
        self.part3json_sol = self.spark.read.option("header", "true").csv("/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/part3jsonsolution")

        
class Part1_Test(PySparkTestCase):
    def test_Part1(self):
        part1("test_part1", "/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/fraud.zip", "fraud")
        part1df = sqlContext.read.format("jdbc").options(
            url='jdbc:mysql://localhost/Al',
            driver='com.mysql.jdbc.Driver',
            dbtable='test_part1',
            user='root',
            password='1234').load()
        part1df.show()
        self.assertEqual(part1df.intersect(self.part1_sol).count(),self.part1_sol.count())
    
    def test_Part1_extra(self):
        part1_extra("test_part1_extra", "/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/fraud.zip", "fraud")
        part1df = sqlContext.read.format("jdbc").options(
            url='jdbc:mysql://localhost/Al',
            driver='com.mysql.jdbc.Driver',
            dbtable='test_part1_extra',
            user='root',
            password='1234').load()
        print("test_Part1_extra")
        self.part1extra_sol.show()
        self.assertEqual(part1df.intersect(self.part1extra_sol).count(),self.part1extra_sol.count())
        
        
class Part2_Test(PySparkTestCase):
    def test_Part2(self):
        #sanitize data here
        part2df = part2("/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/transaction.zip", "transaction")
        self.assertEqual(part2df.intersect(self.part2_sol).count(),self.part2_sol.count())
        
    def test_Part2_extra(self):
        part2df = part2_extra("/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/transaction.zip", "transaction") 
        self.assertEqual(part2df.intersect(self.part2extra_sol).count(),self.part2extra_sol.count())
        
        
class Part3_Test(PySparkTestCase):
    def test_Part3(self):
        #sanitize data here
        part3df, count = part3(self.part1_sol, self.part2_sol)
        self.assertEqual(count,self.part3_sol.count())
    
    def test_Part3_saveJSON(self):
        #sanitize data here
        resultdf = part3_savejson(self.part3json_sol, "/Users/kunalsharma/Desktop/Data-Engineering-Code-Challenge-01-18-2019/testcase/test3.JSON")
        self.assertEqual(resultdf.intersect(self.part3json_sol).count(),self.part3json_sol.count())


In [32]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

File Name                                             Modified             Size
fraud                                          2019-10-25 19:18:58          138
__MACOSX/                                      2019-10-25 19:19:18            0
__MACOSX/._fraud                               2019-10-25 19:18:58          227
+------------------+-------------+
|credit_card_number|         ipv4|
+------------------+-------------+
|      444444444444|192.168.102.7|
|      501850185018|192.168.102.7|
|      510051005100|192.168.102.7|
|      340034003400|192.168.102.7|
+------------------+-------------+

+------------------+-------------+
|credit_card_number|         ipv4|
+------------------+-------------+
|      444444444444|192.168.102.7|
|      501850185018|192.168.102.7|
|      510051005100|192.168.102.7|
|      340034003400|192.168.102.7|
+------------------+-------------+



.

File Name                                             Modified             Size
fraud                                          2019-10-25 19:18:58          138
__MACOSX/                                      2019-10-25 19:19:18            0
__MACOSX/._fraud                               2019-10-25 19:18:58          227
+------------------+-------------+-----+
|credit_card_number|         ipv4|state|
+------------------+-------------+-----+
|      444444444444|192.168.102.7| null|
|      501850185018|192.168.102.7|   WA|
|      510051005100|192.168.102.7| null|
|      340034003400|192.168.102.7|   WA|
+------------------+-------------+-----+

test_Part1_extra
+------------------+-------------+-----+
|credit_card_number|         ipv4|state|
+------------------+-------------+-----+
|      444444444444|192.168.102.7| null|
|      501850185018|192.168.102.7|   WA|
|      510051005100|192.168.102.7| null|
|      340034003400|192.168.102.7|   WA|
+------------------+-------------+-----+



.

File Name                                             Modified             Size
transaction                                    2019-10-25 19:19:04          361
__MACOSX/                                      2019-10-25 19:19:26            0
__MACOSX/._transaction                         2019-10-25 19:19:04          227
+------------------+-------------+-----+
|credit_card_number|         ipv4|state|
+------------------+-------------+-----+
|      444444444444|192.168.102.7|   IA|
|      501850185018|192.168.102.7|   WA|
|      510051005100|192.168.102.7|   IA|
|      340034003400|192.168.102.7|   WA|
|      601160116011|192.168.102.7|   TX|
|      300300300300|192.168.102.7|   WA|
|      350035003500|192.168.102.7|   IA|
|      180018001800|192.168.102.7|   CT|
+------------------+-------------+-----+



.

File Name                                             Modified             Size
transaction                                    2019-10-25 19:19:04          361
__MACOSX/                                      2019-10-25 19:19:26            0
__MACOSX/._transaction                         2019-10-25 19:19:04          227
+------------------+-------------+-----+----------------+
|credit_card_number|         ipv4|state|          vendor|
+------------------+-------------+-----+----------------+
|      444444444444|192.168.102.7|   IA|            Visa|
|      501850185018|192.168.102.7|   WA|         Maestro|
|      510051005100|192.168.102.7|   IA|      Mastercard|
|      340034003400|192.168.102.7|   WA|American Express|
|      601160116011|192.168.102.7|   TX|        Discover|
|      300300300300|192.168.102.7|   WA|     Diners Club|
|      350035003500|192.168.102.7|   IA|           JCB16|
|      180018001800|192.168.102.7|   CT|           JCB15|
|      222222222222|192.168.102.7|   OR|  

..

+------------------+-------------+-----+
|credit_card_number|         ipv4|state|
+------------------+-------------+-----+
|      444444444444|192.168.102.7|   IA|
|      501850185018|192.168.102.7|   WA|
|      510051005100|192.168.102.7|   IA|
|      340034003400|192.168.102.7|   WA|
+------------------+-------------+-----+



.
----------------------------------------------------------------------
Ran 6 tests in 7.937s

OK
